In [2]:
from sys import path
path.append('./C2_code/include/')
import C2_code.include.Node as Node
import C2_code.include.C2_utils as c2u

In [3]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from graphviz import Graph
import time

## Applying external and internal failures. Output: 1) median and average number of survivng connected components (CCs), and the average total number of nodes of a graph, as follows from the ensemble. 2) GCC sizes

In [70]:
Failures = [0.01]#, 0.05, 0.1, 0.15, 0.3]  # CUSTOMISE
Bias = [False]  # CUSTOMISE
MinCCsizes = [1]  # CUSTOMISE
heights = [6, 9, 12]  # CUSTOMISE
CCNos_Ns, GCC_Ns, NintFails = {}, {}, {}

for fails in Failures:
    for bias in Bias:
        TreeEnsemble = c2u.PrunedEnsemble(hmin=6, hmax=13, ExtF=fails, ExtBias=bias)  # CUSTOMISE
        for minccsize in MinCCsizes:
            CCInfo = c2u.CCNosANDSizes(TreeEnsemble[0], MinCC=minccsize)
            GCCs = c2u.GCCSize(TreeEnsemble[0])
            for h in heights:
                barh, Ns, GCCNs = [], [], [] 
                for ell in range(1,h):
                    kwd = 'h=' + str(h) + ', ell=' + str(ell)
                    barh.append( int(np.round(CCInfo[kwd][1])) )
                    Ns.append( CCInfo[kwd][3] )
                    GCCNs.append( GCCs[kwd] )
                kwd = 'h=' + str(h) + '_perc=' + str(fails).split('.')[1] + '_percBias=' + str(bias) \
                + '_MinCCsize=' + str(minccsize)
                CCNos_Ns[kwd] = [barh, Ns]
                GCC_Ns[kwd] = GCCNs
                NintFails[kwd] = TreeEnsemble[1][h]  # keeping track of the internal failures per tree height

/home/stavrogin/Documents/Academics/US_army_corps_engineers/Projects/Decision_theory/Divide_et_impera/C2_code/include/C2_utils.py:53: RuntimeWarning: divide by zero encountered in long_scalars
  G.add_node(child_node.ID, Pknow=1/2**((child_node.highercmnd-1)*np.abs(mes)), ID=child_node.ID,


### Setting up the plots of average number of nodes of graphs and their average number of CCs

In [71]:
# Function to set up the CC No and total N plots versus ell
def C2PercPlots(height, h, kwd):
    ell = list(range(1,h))

    fig, ax1 = plt.subplots(figsize=(10,5))

    color = 'tab:blue'
    ax1.set_xlabel(r'$\ell$', fontsize=25)
    ax1.set_ylabel('CC No', color=color, fontsize=20)  # we already handled the x-label with ax1
    ax1.bar(ell, height[kwd][0], color=color)
    ax1.tick_params(axis='y', labelcolor=color, labelsize=20)
    ax1.tick_params(axis='x', labelsize=20)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:red'
    ax2.set_ylabel('N', color=color, fontsize=20)
    ax2.plot(ell, height[kwd][1], color=color)
    ax2.scatter(ell, height[kwd][1], marker='o', c='r')
    ax2.tick_params(axis='y', labelcolor=color, labelsize=20)

    fig.tight_layout()    

### Setting up the plots of GCC sizes versus $\ell$

In [72]:
# Function to set up the GCC sizes versus ell
def C2GCCPercSizesPlots(height, h, kwd):
    ell = list(range(1,h))

    fig, ax = plt.subplots(figsize=(10,5))

    ax.set_xlabel(r'$\ell$', fontsize=25)
    ax.set_ylabel(r'$N_{GCC}$', fontsize=20)
    ax.plot(ell, height[kwd])
    ax.scatter(ell, height[kwd])
    ax.tick_params(axis='y', labelsize=20)
    ax.tick_params(axis='x', labelsize=20)
    
    fig.tight_layout()

### Setting up the plots of $\tilde{N}_\text{int.}$ versus $\ell$

In [73]:
# Function to set up the internal failure size versus ell
def C2IntFailsPlots(height, h, kwd):
    ell = list(range(1,h))

    fig, ax = plt.subplots(figsize=(10,5))

    ax.set_xlabel(r'$\ell$', fontsize=25)
    ax.set_ylabel(r'$\tilde{N}_{int.}$', fontsize=20)
    ax.plot(ell, height[kwd])
    ax.scatter(ell, height[kwd])
    ax.tick_params(axis='y', labelsize=20)
    ax.tick_params(axis='x', labelsize=20)
    
    fig.tight_layout()

#### Saving the data in files

In [74]:
path = './Manuscript/Images/'
for minccsize in MinCCsizes:
    for bias in Bias:
        for h in heights:
            for fails in Failures: 
                kwd = 'h=' + str(h) + '_perc=' + str(fails).split('.')[1] + '_percBias=' + str(bias) \
                + '_MinCCsize=' + str(minccsize)
                ell = list(range(1,h))
                if minccsize == 1:
                    if bias:
                        Temp = np.array([ell, CCNos_Ns[kwd][0], CCNos_Ns[kwd][1]]).T
                        pd.DataFrame(Temp, columns=['ell', 'CCs', 'N']).to_csv(path + 'MinCCSize_1/Non_Uniform_perc/' + kwd + '.csv')
                        Temp2 = np.array( [ell, GCC_Ns[kwd]] ).T
                        pd.DataFrame(Temp2, columns=['ell', 'NGCC']).to_csv(path + 'MinCCSize_1/Non_Uniform_perc/GCC_' + kwd + '.csv')
                        Temp3 = np.array( [ell, NintFails[kwd]] ).T
                        pd.DataFrame(Temp3, columns=['ell', 'Nint']).to_csv(path + 'MinCCSize_1/Non_Uniform_perc/Nint_' + kwd + '.csv')                        
                    else:
                        Temp = np.array([ell, CCNos_Ns[kwd][0], CCNos_Ns[kwd][1]]).T
                        pd.DataFrame(Temp, columns=['ell', 'CCs', 'N']).to_csv(path + 'MinCCSize_1/Uniform_perc/' + kwd + '.csv')
                        Temp2 = np.array( [ell, GCC_Ns[kwd]] ).T
                        pd.DataFrame(Temp2, columns=['ell', 'NGCC']).to_csv(path + 'MinCCSize_1/Uniform_perc/GCC_' + kwd + '.csv')
                        Temp3 = np.array( [ell, NintFails[kwd]] ).T
                        pd.DataFrame(Temp3, columns=['ell', 'Nint']).to_csv(path + 'MinCCSize_1/Uniform_perc/Nint_' + kwd + '.csv')
                elif minccsize == 3:
                    if bias:
                        Temp = np.array([ell, CCNos_Ns[kwd][0], CCNos_Ns[kwd][1]]).T
                        pd.DataFrame(Temp, columns=['ell', 'CCs', 'N']).to_csv(path + 'MinCCSize_3/Non_Uniform_perc/' + kwd + '.csv')
                        Temp2 = np.array( [ell, GCC_Ns[kwd]] ).T
                        pd.DataFrame(Temp2, columns=['ell', 'NGCC']).to_csv(path + 'MinCCSize_3/Non_Uniform_perc/GCC_' + kwd + '.csv')
                        Temp3 = np.array( [ell, NintFails[kwd]] ).T
                        pd.DataFrame(Temp3, columns=['ell', 'Nint']).to_csv(path + 'MinCCSize_3/Non_Uniform_perc/Nint_' + kwd + '.csv')
                    else:
                        Temp = np.array([ell, CCNos_Ns[kwd][0], CCNos_Ns[kwd][1]]).T
                        pd.DataFrame(Temp, columns=['ell', 'CCs', 'N']).to_csv(path + 'MinCCSize_3/Uniform_perc/' + kwd + '.csv')
                        Temp2 = np.array( [ell, GCC_Ns[kwd]] ).T
                        pd.DataFrame(Temp2, columns=['ell', 'NGCC']).to_csv(path + 'MinCCSize_3/Uniform_perc/GCC_' + kwd + '.csv')
                        Temp3 = np.array( [ell, NintFails[kwd]] ).T
                        pd.DataFrame(Temp3, columns=['ell', 'Nint']).to_csv(path + 'MinCCSize_3/Uniform_perc/Nint_' + kwd + '.csv')
                else:
                    if bias:
                        Temp = np.array([ell, CCNos_Ns[kwd][0], CCNos_Ns[kwd][1]]).T
                        pd.DataFrame(Temp, columns=['ell', 'CCs', 'N']).to_csv(path + 'MinCCSize_5/Non_Uniform_perc/' + kwd + '.csv')
                        Temp2 = np.array( [ell, GCC_Ns[kwd]] ).T
                        pd.DataFrame(Temp2, columns=['ell', 'NGCC']).to_csv(path + 'MinCCSize_5/Non_Uniform_perc/GCC_' + kwd + '.csv')
                        Temp3 = np.array( [ell, NintFails[kwd]] ).T
                        pd.DataFrame(Temp3, columns=['ell', 'Nint']).to_csv(path + 'MinCCSize_5/Non_Uniform_perc/Nint_' + kwd + '.csv')
                    else:
                        Temp = np.array([ell, CCNos_Ns[kwd][0], CCNos_Ns[kwd][1]]).T
                        pd.DataFrame(Temp, columns=['ell', 'CCs', 'N']).to_csv(path + 'MinCCSize_5/Uniform_perc/' + kwd + '.csv')
                        Temp2 = np.array( [ell, GCC_Ns[kwd]] ).T
                        pd.DataFrame(Temp2, columns=['ell', 'NGCC']).to_csv(path + 'MinCCSize_5/Uniform_perc/GCC_' + kwd + '.csv')
                        Temp3 = np.array( [ell, NintFails[kwd]] ).T
                        pd.DataFrame(Temp3, columns=['ell', 'Nint']).to_csv(path + 'MinCCSize_5/Uniform_perc/Nint_' + kwd + '.csv')

#### Reading the data from files

In [75]:
path = './Manuscript/Images/'
for minccsize in MinCCsizes:
    for bias in Bias:
        for h in heights:
            for fails in Failures: 
                kwd = 'h=' + str(h) + '_perc=' + str(fails).split('.')[1] + '_percBias=' + str(bias) \
                + '_MinCCsize=' + str(minccsize)
                temp = pd.read_csv(path + 'MinCCSize_1/Uniform_perc/' + kwd + '.csv', usecols=['CCs', 'N'])
                CCNos_Ns[kwd] = []
                CCNos_Ns[kwd].append( temp['CCs'].values )
                CCNos_Ns[kwd].append( temp['N'].values )
                GCC_Ns[kwd] = pd.read_csv(path + 'MinCCSize_1/Uniform_perc/GCC_' + kwd + '.csv', usecols=['NGCC'])
                NintFails[kwd] = pd.read_csv(path + 'MinCCSize_1/Uniform_perc/Nint_' + kwd + '.csv', usecols=['Nint'])

#### Saving the GCC vs $\ell$ and CC Nos & N vs $\ell$ plots in designated locations

In [76]:
path = './Manuscript/Images/'
for minccsize in MinCCsizes:
    for bias in Bias:
        for h in heights:
            for fails in Failures: 
                kwd = 'h=' + str(h) + '_perc=' + str(fails).split('.')[1] + '_percBias=' + str(bias) \
                + '_MinCCsize=' + str(minccsize)
                if minccsize == 1:
                    if bias:
                        C2PercPlots(CCNos_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_1/Non_Uniform_perc/' + kwd)
                        plt.close()
                        C2GCCPercSizesPlots(GCC_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_1/Non_Uniform_perc/GCC_' + kwd)
                        plt.close()
                        C2IntFailsPlots(NintFails, h, kwd)
                        plt.savefig(path + 'MinCCSize_1/Non_Uniform_perc/Nint_' + kwd)
                        plt.close()
                    else:
                        C2PercPlots(CCNos_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_1/Uniform_perc/' + kwd)
                        plt.close()
                        C2GCCPercSizesPlots(GCC_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_1/Uniform_perc/GCC_' + kwd)
                        plt.close()
                        C2IntFailsPlots(NintFails, h, kwd)
                        plt.savefig(path + 'MinCCSize_1/Uniform_perc/Nint_' + kwd)
                        plt.close()
                elif minccsize == 3:
                    if bias:
                        C2PercPlots(CCNos_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_3/Non_Uniform_perc/' + kwd)
                        plt.close()
                        C2GCCPercSizesPlots(GCC_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_3/Non_Uniform_perc/GCC_' + kwd)
                        plt.close()
                        C2IntFailsPlots(NintFails, h, kwd)
                        plt.savefig(path + 'MinCCSize_3/Non_Uniform_perc/Nint_' + kwd)
                        plt.close()                                                
                    else:
                        C2PercPlots(CCNos_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_3/Uniform_perc/' + kwd)
                        plt.close()
                        C2GCCPercSizesPlots(GCC_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_3/Uniform_perc/GCC_' + kwd)
                        plt.close()
                        C2IntFailsPlots(NintFails, h, kwd)
                        plt.savefig(path + 'MinCCSize_3/Uniform_perc/Nint_' + kwd)
                        plt.close()                                                
                else:
                    if bias:
                        C2PercPlots(CCNos_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_5/Non_Uniform_perc/' + kwd)
                        plt.close()
                        C2GCCPercSizesPlots(GCC_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_5/Non_Uniform_perc/GCC_' + kwd)
                        plt.close()
                        C2IntFailsPlots(NintFails, h, height)
                        plt.savefig(path + 'MinCCSize_5/Non_Uniform_perc/Nint_' + kwd)
                        plt.close()
                    else:
                        C2PercPlots(CCNos_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_5/Uniform_perc/' + kwd)
                        plt.close()
                        C2GCCPercSizesPlots(GCC_Ns, h, kwd)
                        plt.savefig(path + 'MinCCSize_5/Uniform_perc/GCC_' + kwd)
                        plt.close()
                        C2IntFailsPlots(NintFails, h, kwd)
                        plt.savefig(path + 'MinCCSize_5/Uniform_perc/Nint_' + kwd)
                        plt.close()